<a href="https://colab.research.google.com/github/CP2J/cp2j/blob/ACJ-9-MF-SGD-/MF_SGD(study).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import files
from collections import Counter
from sklearn.model_selection import train_test_split
from scipy import sparse

In [2]:
from google.colab import drive
drive.mount('/content/drive')
rating = pd.read_csv('/content/drive/MyDrive/ml-100k/u.data', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])

Mounted at /content/drive


# MF-SGD (Matrix Factorization - Stochastic Gradient Descent)
* SVD에서 쓰인 행렬분해를 이용, 확률적 경사하강 기법으로 오차를 줄이는 방향으로 학습한다.
* SVD의 행렬분해에서는 null값이 존재하면 안되기에 평균값, 최빈값 등을 사용했으나 여기서는 랜덤값 지정 후 오차를 줄이는 방향으로 학습.
* 결국 데이터가 sparse 할 수록 임의값에 의존하던 이전 모델들에 비해 성능이 더 잘 나오게 된다.

# 0410 추가본 - 다른 레퍼런스 참고
 https://big-dream-world.tistory.com/69

1. 분해한 P, Q 행렬 임의값으로 생성
2. P 행렬, Q 전치행렬 곱해서 예측행렬 생성, 실제 R 행렬과 차이 계산(R 행렬 내 존재하는 실제값들과의 차이) 
3. 차이 줄이는 방향으로 P, Q 행렬 업데이트
4. 반복하며 근사화


  ** 상단 코드와의 차이  
1. train_test_split 안함 - train값으로 안본 영화 평점 예측 목적
2. 코드 간소화
3. 추후 수정

In [3]:
rating.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
rating_df = rating.pivot(index = 'user_id', columns = 'item_id', values = 'rating')
real_mat = rating_df.to_numpy()
num_user, num_item = real_mat.shape

In [5]:
real_mat

array([[ 5.,  3.,  4., ..., nan, nan, nan],
       [ 4., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [ 5., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan,  5., nan, ..., nan, nan, nan]])

In [6]:
print(num_user, num_item)

943 1682


In [7]:
# P, Q 만들기
K = 5
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_user, K))
Q = np.random.normal(scale=1./K, size=(num_item, K))
print(P.shape, Q.shape)

(943, 5) (1682, 5)


In [8]:
P

array([[ 0.32486907, -0.12235128, -0.10563435, -0.21459372,  0.17308153],
       [-0.46030774,  0.34896235, -0.15224138,  0.06380782, -0.04987408],
       [ 0.29242159, -0.41202814, -0.06448344, -0.07681087,  0.22675389],
       ...,
       [ 0.25492871, -0.05039558,  0.097754  ,  0.07813002,  0.15870667],
       [-0.00779446,  0.21716811, -0.10508495,  0.21014734,  0.24452006],
       [-0.0066753 ,  0.09060358,  0.01855727,  0.119657  ,  0.13527794]])

In [9]:
P.max()

0.7917205408075927

In [10]:
Q

array([[ 0.05209434, -0.04480388,  0.16500092,  0.09246273,  0.47655322],
       [-0.21540172,  0.38926981,  0.08383212,  0.161059  , -0.12298153],
       [-0.08035176,  0.07262364,  0.00623031, -0.27591379,  0.01610528],
       ...,
       [ 0.00449204,  0.00646432, -0.3573115 ,  0.21257792, -0.12423349],
       [ 0.01938678,  0.09546832,  0.06378416, -0.04559171, -0.2420766 ],
       [ 0.23471571, -0.58564795, -0.0952186 , -0.26388873, -0.07425806]])

In [11]:
Q.max()

0.8053698089094756

In [12]:
non_zeros = [(i, j, real_mat[i, j]) for i in range(num_user) for j in range(num_item) if real_mat[i, j] > 0]
# 평점이 있는 user 위치, item 위치, rating 값 튜플로 묶어 리스트 내 저장
non_zeros

[(0, 0, 5.0),
 (0, 1, 3.0),
 (0, 2, 4.0),
 (0, 3, 3.0),
 (0, 4, 3.0),
 (0, 5, 5.0),
 (0, 6, 4.0),
 (0, 7, 1.0),
 (0, 8, 5.0),
 (0, 9, 3.0),
 (0, 10, 2.0),
 (0, 11, 5.0),
 (0, 12, 5.0),
 (0, 13, 5.0),
 (0, 14, 5.0),
 (0, 15, 5.0),
 (0, 16, 3.0),
 (0, 17, 4.0),
 (0, 18, 5.0),
 (0, 19, 4.0),
 (0, 20, 1.0),
 (0, 21, 4.0),
 (0, 22, 4.0),
 (0, 23, 3.0),
 (0, 24, 4.0),
 (0, 25, 3.0),
 (0, 26, 2.0),
 (0, 27, 4.0),
 (0, 28, 1.0),
 (0, 29, 3.0),
 (0, 30, 3.0),
 (0, 31, 5.0),
 (0, 32, 4.0),
 (0, 33, 2.0),
 (0, 34, 1.0),
 (0, 35, 2.0),
 (0, 36, 2.0),
 (0, 37, 3.0),
 (0, 38, 4.0),
 (0, 39, 3.0),
 (0, 40, 2.0),
 (0, 41, 5.0),
 (0, 42, 4.0),
 (0, 43, 5.0),
 (0, 44, 5.0),
 (0, 45, 4.0),
 (0, 46, 4.0),
 (0, 47, 5.0),
 (0, 48, 3.0),
 (0, 49, 5.0),
 (0, 50, 4.0),
 (0, 51, 4.0),
 (0, 52, 3.0),
 (0, 53, 3.0),
 (0, 54, 5.0),
 (0, 55, 4.0),
 (0, 56, 5.0),
 (0, 57, 4.0),
 (0, 58, 5.0),
 (0, 59, 5.0),
 (0, 60, 4.0),
 (0, 61, 3.0),
 (0, 62, 2.0),
 (0, 63, 5.0),
 (0, 64, 4.0),
 (0, 65, 4.0),
 (0, 66, 3.0),
 (0, 

In [13]:
from sklearn.metrics import mean_squared_error

def get_rmse(real_mat, P, Q, non_zeros):
  # real_df = 실제 유저 아이템 행렬
  # P, Q = 잠재요인, user와 item으로 분해된 잠재행렬, 이걸로 예측 행렬 생성
  # non_null = real_df 내 null값 아니었던 것(real_df 만으로 함수 내 해결할 수 있을것으로 보이나, 저장해서 반복 계산 시 효율성 재고)
  error = 0
  pred_mat = np.dot(P, Q.T)
  # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
  user_non_zero_index = [non_zero[0] for non_zero in non_zeros]
  item_non_zero_index = [non_zero[1] for non_zero in non_zeros]
  real_mat_non_zero = real_mat[user_non_zero_index, item_non_zero_index]
  pred_mat_non_zero = pred_mat[user_non_zero_index, item_non_zero_index]

  mse = mean_squared_error(real_mat_non_zero, pred_mat_non_zero)
  rmse = np.sqrt(mse)

  return rmse

In [14]:
get_rmse(real_mat, P,Q, non_zeros)

3.7060615755099544

**비용 함수 생략 잠재 행렬(P, Q) 내 벡터(Pu, Qi) 업데이트 식**  
  
* Rui = 실제 행렬의 (u, i) 값  
* R^ui = 예측 행렬의 (u, i) 값  
* Eui = (u, i)위치의 실제행렬 값 예측행렬 값 차이 

  * $e_{ui}=r_{ui}-p_uq_i^T$
* Gradient
  * $\frac {\partial L}{\partial p_u} = \frac {\partial(r_{ui}-p_uq_i^T)^2}{\partial p_u} + \frac {\partial\lambda||p_u||^2_2}{\partial p_u} = -2(r_{ui}-p_uq_i^T)q_i+2\lambda p_u = -2(e_{ui}q_i-\lambda p_u)$
    * 기존 LOSS에서 $p_u$에 대해 편미분을 진행해 필요없는 $q_i$항을 모두 지우고 계산하면 $-2(e_{ui}q_i-\lambda p_u)$가 남게 됨.
  * Gradient 반대로 $p_u, q_i$ 업데이트
    * $p_u ← p_u +\eta \cdot(e_{ui}q_i - \lambda p_u)$
    * $q_i ← q_i +\eta \cdot(e_{ui}p_u - \lambda q_i)$
  
* Pu(new) = Pu + 학습률 * (Eui * Qi - lambda(L2 정규화 계수) * Pu)  
* Qi(new) = Qi + 학습률 * (Eui * Pu - lambda(L2 정규화 계수) * Qi)


In [15]:
iteration = 2000
learning_rate = 0.001
lmbda = 0.01

In [16]:
for iter in range(iteration):
  for i, j, Rij in non_zeros:
    # 실제 값과 예측 값의 차이인 오류 값 구함
    Eij = Rij - np.dot(P[i, :], Q[j, :].T)
    # 벡터 업데이트 :Regularization을 반영한 SGD 업데이트 공식 적용
    P[i, :] = P[i, :] + learning_rate *(Eij* Q[j, :] - lmbda* P[i, :])
    Q[j, :] = Q[j, :] + learning_rate *(Eij* P[i, :] - lmbda* Q[j, :])
  rmse = get_rmse(real_mat, P, Q, non_zeros)
  if (iter+1) % 10 == 0:
    print('iteration num : ', iter+1, " rmse : ", rmse)

iteration num :  10  rmse :  1.7554530700155224
iteration num :  20  rmse :  1.079547313608221
iteration num :  30  rmse :  0.9772009392521711
iteration num :  40  rmse :  0.9423830292652029
iteration num :  50  rmse :  0.923054903690508
iteration num :  60  rmse :  0.9087425845002808
iteration num :  70  rmse :  0.8967946490366422
iteration num :  80  rmse :  0.8865177108755139
iteration num :  90  rmse :  0.8775954095775489
iteration num :  100  rmse :  0.8697653141003603
iteration num :  110  rmse :  0.86282184648358
iteration num :  120  rmse :  0.8566197152110971
iteration num :  130  rmse :  0.851057045541995
iteration num :  140  rmse :  0.8460580933643697
iteration num :  150  rmse :  0.8415617138768727
iteration num :  160  rmse :  0.8375151242732989
iteration num :  170  rmse :  0.83387102905159
iteration num :  180  rmse :  0.8305864519363927
iteration num :  190  rmse :  0.8276222839910465
iteration num :  200  rmse :  0.8249430733695061
iteration num :  210  rmse :  0.8225

KeyboardInterrupt: ignored

이후 내용은 추천시스템.  
test train 나눠서 검증이 아니라 최대한 오차를 낮춘 다음 유저가 안 본 영화 중 예측 평점 높은 순대로 추천하는 시스템.  
그렇다면 과적합 문제가 발생하지 않을까?  
